In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from FeatureExtractor import *

In [2]:
stack = 'MD594'

sec = 145
dm = DataManager(stack=stack, section=sec, segm_params_id='tSLIC200', load_mask=False)

dm._load_image(['rgb'])

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2224: DecompressionBombWarning: Image size (321922048 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [35]:
patch_size = 224
half_size = patch_size/2
stride = 56

h, w = dm.image_rgb.shape[:2]

ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                     indexing='xy')
    
sample_locations = np.c_[xs.flat, ys.flat]

In [7]:
# mask = imread(dm.data_dir+'/%(stack)s_thumbnail_aligned_cropped_mask/%(stack)s_%(slice_str)s_thumbnail_aligned_cropped_mask.png' % \
#        {'stack': dm.stack, 'slice_str': dm.slice_str}) > 0
mask = dm.load_thumbnail_mask()
mask_lossless = rescale(dm.thumbmail_mask, 32)

In [31]:
patches = []

for x, y in sample_locations:

    patch = dm.image_rgb[y-half_size:y+half_size, x-half_size:x+half_size]
    patches.append(patch)
        
print len(patches), 'patches'

100224 patches


In [32]:
output_dir = '/home/yuncong/CSHL_data_patches/'+ stack +'_patches/%04d_reduced'%sec
if not os.path.exists(output_dir):
    os.makedirs(output_dir)    

In [37]:
for i, p in enumerate(patches):
    x, y = sample_locations[i]
    if mask_lossless[y,x]:
        cv2.imwrite(output_dir + '/%08d.tif'%i, img_as_ubyte(p))

KeyboardInterrupt: 

In [ ]:
sample_locations_reduced = []
for i, p in enumerate(patches):
    c, r = sample_locations[i]
    if r > 5984 and r < 5984+4944 and c > 2240 and c < 2240+7280:
        if mask_lossless[c,r]:
            sample_locations_reduced.append((c,r))
            cv2.imwrite(output_dir + '/%08d.jpg'%i, img_as_ubyte(p))

In [ ]:
np.save('/home/yuncong/CSHL_data_patches/MD594_sample_locations_reduced.npy', sample_locations_reduced)